In [ ]:
#### In router agent allow multiple async calls to top-k agents
#### Components - Deterministic or Probabilistic
#### Add timeout to agent calls
#### Tools and components are very similar but components are what we control and tools are external services

#### Add ability to call any Rest API as a tool: URL, input, output, types of input and output. So we can
#### connect to any external service. For example, we can connect to an API to send out prompt to evaluate it.
#### Command agent can call any agent, tool, or component. It can also call a router agent to call multiple agents.
#### Command agent should be able to handle all types of inputs and outputs. Text, Image and Voice.
#### Have the ability to expose the various parts of the Command agent as a REST API.
#### Add logging and monitoring to the agent. Log all the inputs and outputs. Log the time taken for each call.
#### Example types to be implemented: CX, Synthetic data generation (contract, customer chat generation, csv data), Media planning, Invoice extraction,
#### Create a generic API calling tool.
#### Add a validation tool to validate the output of the agent. For example, if the agent is supposed to return a number, then validate that the output is a number. Also verify that the variables are sent back to the agents correctly. And the models are not hallucinating.

### Function schema decorator


In [ ]:
import inspect
import json
from typing import Any, Dict, get_type_hints, List, Union, Optional

from nltk.inference.mace import arguments


def function_schema(func):
    """
    Decorator that generates an OpenAI function schema and attaches it to the function.
    """
    def python_function_to_openai_schema(func) -> Dict[str, Any]:
        """Converts a function into an OpenAI JSON schema."""
        signature = inspect.signature(func)
        type_hints = get_type_hints(func)

        schema = {
            "type": "function",
            "function":{
            "name": func.__name__,
            "description": func.__doc__ or f"Function {func.__name__}",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": []
            }
        }
        }

        for param_name, param in signature.parameters.items():
            param_type = type_hints.get(param_name, Any)
            openai_type, is_optional = python_type_to_openai_type(param_type)

            # schema["parameters"]["properties"][param_name] = {
            schema["function"]["parameters"]["properties"][param_name] = {
                "type": openai_type,
                "description": f"{param_name} parameter"
            }

            # Add to required list only if it's not Optional and has no default value
            if not is_optional and param.default is inspect.Parameter.empty:
                # schema["parameters"]["required"].append(param_name)
                schema["function"]["parameters"]["required"].append(param_name)

        return schema

    def python_type_to_openai_type(py_type) -> (str, bool):
        """Maps Python types to OpenAI JSON schema types, supporting Optional and List."""
        from typing import get_origin, get_args

        # Handle Optional[X] (Union[X, None])
        if get_origin(py_type) is Union:
            args = get_args(py_type)
            non_none_types = [t for t in args if t is not type(None)]
            if len(non_none_types) == 1:
                openai_type, _ = python_type_to_openai_type(non_none_types[0])
                return openai_type, True  # It's Optional

        # Handle List[X]
        if get_origin(py_type) is list or get_origin(py_type) is List:
            return "array", False

        # Base type mapping
        mapping = {
            int: "integer",
            float: "number",
            str: "string",
            bool: "boolean",
            dict: "object",
        }

        return mapping.get(py_type, "string"), False  # Default to string

    # Attach schema as a function attribute
    func.openai_schema = python_function_to_openai_schema(func)
    return func



### Logging decorator

In [ ]:
import functools
from typing import Any, Callable

def log_execution(func: Callable) -> Callable:
    """Decorator to log the input and output of a function."""
    @functools.wraps(func)
    def wrapper(self, *args, **kwargs) -> Any:
        self.logs["input"].append(kwargs)
        try:
            result = func(self, *args, **kwargs)
            self.logs["output"].append(result)
            return result
        except Exception as e:
            self.logs["output"].append(f"Error: {e}")
            raise
    return wrapper

### OpenAI Credentials and Client

In [ ]:
import openai
import dotenv
import os

dotenv.load_dotenv(".env.local")
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

CX_ID = os.getenv("CX_ID_PERSONAL")
GOOGLE_API = os.getenv("GOOGLE_API_PERSONAL")


### Tools

In [ ]:
import requests
from bs4 import BeautifulSoup
import markdownify


@function_schema
def add_numbers(a: int, b: int) -> str:
    """
    Adds two numbers and returns the sum.
    """
    return f"The sum of {a} and {b} is {a + b}"

@function_schema
def get_customer_order(customer_id: int) -> str:
    """"
    Returns the order number for a given customer ID."""
    if customer_id == 1111:
        return f"The order number for customer ID {customer_id} is 1720"
    elif customer_id == 2222:
        return f"The order number for customer ID {customer_id} is 9"
    return f"No order found for customer ID {customer_id}"

@function_schema
def weather_forecast(location: str) -> str:
    """"
    Returns the weather forecast for a given location.
    """
    return "It's sunny" if "a" in location else "It's rainy"

@function_schema
def url_to_markdown(url):
    try:
        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')
        content = soup.find('body')

        if content:
            markdown_content = markdownify.markdownify(str(content), heading_style="ATX")
            return markdown_content[:20000]
        else:
            return "No content found in the webpage body."

    except requests.RequestException as e:
        return f"Error fetching URL: {e}"

@function_schema
def web_search(query: str,num: Optional[int]=2) -> str:
    """
    You can use this tool to get any information from the web. Just type in your query and get the results.
    """
    num=1
    # try:
    url = f"https://www.googleapis.com/customsearch/v1?q={query}&key={GOOGLE_API}&cx={CX_ID}&num={num}"
    response = requests.get(url)
    print(response)
    urls = [i["link"] for i in response.json()["items"]]
    print(urls)
    text = "\n".join([url_to_markdown(i) for i in urls])
    # print(text)
    prompt = f"Use the following text to answer the given: {query} \n\n ---BEGIN WEB TEXT --- {text} ---BEGIN WEB TEXT --- "
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": f"You're a web search agent."},{"role": "user", "content": prompt}],
    )
    return response.choices[0].message.content

    # return f"""
    # Here are the web results for your query: {response.choices[0].message.content}
    # """
    # except:
    #     return "Error fetching search results."


In [ ]:
web_search.openai_schema

In [ ]:
tool_store = {
    "add_numbers": add_numbers,
    "weather_forecast": weather_forecast,
    "url_to_markdown": url_to_markdown,
    "web_search": web_search,
    "get_customer_order": get_customer_order
}

### Components

In [ ]:
from pydantic import BaseModel, Field
from typing import Optional, Dict, List, Any

class Component(BaseModel):
    name: str
    description: Optional[str]
    input_type: Optional[str] = "text"
    output_type: Optional[str] = "text"
    parameters: Dict[str, Any]
    logs: Dict[str, List] = Field(default_factory=lambda: {"input": [], "output": []})

    @log_execution
    def execute(self, **kwargs) -> Any:
        """Execution script for each component."""
        raise NotImplementedError("Component execution logic should be implemented in subclasses.")

class WebSearchComponent(Component):
    search_engine: Optional[str] = "google"

    @log_execution
    def execute(self, query: str) -> Any:
        """Performs a web search using Google or Bing API."""
        return web_search(query)

### Testing the WebSearchComponent

In [ ]:
search_component = WebSearchComponent(name= "WebSearch",
                   description= "Search the web for information.",
                    search_engine="google",
                   parameters={"query": "string"})

In [ ]:
# search_component.execute(query="What is the latest news on Toshiba?")
pass

In [ ]:
# print(search_component.logs["output"][0])

### Prompt Object

In [ ]:
import datetime
from typing import Optional, Dict, List
import uuid
import pydantic

class PromptReadListRequest(pydantic.BaseModel):
    app_name: str
    prompt_label: str

class PromptReadDeployedRequest(pydantic.BaseModel):
    app_name: str
    prompt_label: str

class PromptReadRequest(pydantic.BaseModel):
    app_name: str
    pid: uuid.UUID

class PromptObjectRequest(pydantic.BaseModel):
    """
        pid: randomUUID(),
        app_name: appName,
        vendor: selectedVendor,
        model_name: selectedModelName,
        prompt_label: selectedPromptLabel,
        prompt: selectedPrompt,
        hash: hash,
        temperature: temperature,
        max_tokens: maxTokens,
        version: version,
        is_deployed: false,
        deployed_time: new Date().toISOString(),
    """
    pid: uuid.UUID
    is_system_prompt: Optional[bool]
    app_name: str
    vendor: str
    model_name: str
    prompt_label: str
    prompt: str
    hash: str
    temperature: str
    max_tokens: str
    version: Optional[str]
    tags: Optional[List[str]]
    is_deployed: Optional[bool]
    hyper_parameters: Optional[Dict[str, str]]
    variables: Optional[List[str]]
    deployed_time: Optional[datetime.datetime]

class PromptDeployRequest(pydantic.BaseModel):
    """
        pid: randomUUID(),
        app_name: appName,
    """
    pid: uuid.UUID
    app_name: str


class PromptObject(pydantic.BaseModel):
    pid: uuid.UUID
    prompt_label: str
    prompt: str
    sha_hash: str
    uniqueLabel:str
    appName:str
    version:str
    createdTime:datetime.datetime
    deployedTime:Optional[datetime.datetime]
    last_deployed:Optional[datetime.datetime]
    modelProvider: str
    modelName: str
    isDeployed:bool
    tags: Optional[List[str]]
    hyper_parameters: Optional[Dict[str, str]]
    variables: Optional[Dict[str, str]]



### Agent

In [ ]:
from typing import List, Dict, Any, Optional, Literal
from pydantic import BaseModel
from datetime import datetime
import uuid



class Agent(BaseModel):
    name: str
    agent_id: uuid.UUID
    parent_agent: Optional[uuid.UUID] = None
    system_prompt: PromptObject
    persona: Optional[str]
    functions: List[Dict[str, Any]]
    routing_options: Dict[str, str]
    short_term_memory: bool = False
    long_term_memory: bool = False
    reasoning: bool = False
    # input_type: Optional[Literal["text", "voice", "image"]] = "text"
    # output_type: Optional[Literal["text", "voice", "image"]] = "text"
    # Make input output type a list of types
    input_type: Optional[List[Literal["text", "voice", "image"]]] = ["text", "voice"]
    output_type: Optional[List[Literal["text", "voice", "image"]]] = ["text", "voice"]
    response_type: Optional[Literal["json", "yaml", "markdown", "HTML", "None"]] = "json"
    # agent_type: Optional[Literal["agent", "workflow"]] = "agent"

    # Execution parameters
    max_retries: int = 3
    timeout: Optional[int] = None
    deployed: bool = False
    status: Literal["active", "paused", "terminated"] = "active"
    priority: Optional[int] = None

    # Agent, Human or Parent Agent
    failure_strategies: Optional[List[str]]

    # Logging and monitoring
    session_id: Optional[str] = None
    last_active: Optional[datetime] = None
    # logging_level: Optional[Literal["debug", "info", "warning", "error"]] = "info"  # Debug level

    collaboration_mode: Optional[Literal["single", "team", "parallel", "sequential"]] = "single"  # Multi-agent behavior

    # Add a function to import the right prompt for the agent.
    def execute(self, **kwargs) -> Any:
        """Execution script for each component."""
        raise NotImplementedError("Component execution logic should be implemented in subclasses.")




In [ ]:
agent_system_prompt = PromptObject(pid=uuid.uuid4(),
                                             prompt_label="Stock Analytics",
                                             prompt="You're a helpful assistant.",
                                             sha_hash="000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8c",
                                             uniqueLabel="StockAnalytics",
                                             appName="Berkshire Hathaway",
                                             version="1.0",
                                             createdTime=datetime.now(),
                                             deployedTime=None,
                                             last_deployed=None,
                                             modelProvider="OpenAI",
                                             modelName="GPT-4o-mini",
                                             isDeployed=False,
                                             tags=["search", "web"],
                                             hyper_parameters={"temperature": "0.7"},
                                             variables={"search_engine": "google"})

### Agent Schema

In [ ]:
# Agent Schema function

import inspect
import json
from typing import Any, Dict, List, Union, get_type_hints

def agent_schema(cls):
    """
    Decorator that generates an OpenAI function schema for an agent and attaches it to the class.
    The tool name will be the agent class name, not the 'execute' function.
    """
    def python_function_to_openai_schema(agent_cls) -> Dict[str, Any]:
        """Converts an agent class into an OpenAI JSON schema based on its `execute` method."""
        execute_method = getattr(agent_cls, "execute", None)
        if execute_method is None:
            raise ValueError(f"{agent_cls.__name__} must have an 'execute' method.")

        signature = inspect.signature(execute_method)
        type_hints = get_type_hints(execute_method)

        schema = {
            "type": "function",
            "function": {
                "name": agent_cls.__name__,  # Tool name = Class name
                "description": execute_method.__doc__ or f"Agent {agent_cls.__name__} execution function",
                "parameters": {
                    "type": "object",
                    "properties": {},
                    "required": []
                }
            }
        }

        for param_name, param in signature.parameters.items():
            if param_name == "self":
                continue  # Skip 'self' parameter

            param_type = type_hints.get(param_name, Any)
            openai_type, is_optional = python_type_to_openai_type(param_type)

            schema["function"]["parameters"]["properties"][param_name] = {
                "type": openai_type,
                "description": f"{param_name} parameter"
            }

            if not is_optional and param.default is inspect.Parameter.empty:
                schema["function"]["parameters"]["required"].append(param_name)

        return schema

    def python_type_to_openai_type(py_type) -> (str, bool):
        """Maps Python types to OpenAI JSON schema types, handling Optional and List types."""
        from typing import get_origin, get_args

        if get_origin(py_type) is Union:
            args = get_args(py_type)
            non_none_types = [t for t in args if t is not type(None)]
            if len(non_none_types) == 1:
                openai_type, _ = python_type_to_openai_type(non_none_types[0])
                return openai_type, True  # It's Optional

        if get_origin(py_type) is list or get_origin(py_type) is List:
            return "array", False

        mapping = {
            int: "integer",
            float: "number",
            str: "string",
            bool: "boolean",
            dict: "object",
        }

        return mapping.get(py_type, "string"), False  # Default to string

    # Attach schema as a class attribute
    cls.openai_schema = python_function_to_openai_schema(cls)
    return cls



In [ ]:
@agent_schema
class WebAgent(Agent):
    def execute(self, query: Any) -> Any:
        """
        Ask the agent anything related to arithmetic or web search and it will try to answer it.
        """
        tries = 0
        system_prompt = self.system_prompt.prompt + f"""
        Here are the routing options:
        {"\n".join([f"{k}: {v}" for k, v in self.routing_options.items()])}

        Your response should be in the format:
        {{ "routing": "respond", "content": "The answer to the query."}}

        """
        messages=[{"role": "system", "content": system_prompt},{"role": "user", "content": query}]

        while tries < self.max_retries:
            print(tries)
            print(messages)
            try:
                response = client.chat.completions.create(
                    model="gpt-4o-mini",
                    messages=messages,
                    functions=self.functions,
                    # tools=self.functions,
                    # parallel_tool_calls=True,
                )
                print(response)
                if response.choices[0].message.function_call:
                    print(response.choices[0].message.function_call)
                    # print(**response.choices[0].message.function_call.arguments)
                    arguments = json.loads(response.choices[0].message.function_call.arguments)
                    function_name = response.choices[0].message.function_call.name
                    output = tool_store[function_name](**arguments)
                    # output = "Toshiba is a great company"
                    # print(output)
                    messages+=[{"role":"system", "content": f"I called the function {function_name} with the arguments {arguments} and got the output {output}."},
                        {"role": "user", "content": """Now, try to answer the original query given the output. If you can't, try using another tool. If it does answer the query, you can stop here and just answer the query. If you can't then you can give up.
                    """}]
                else:
                    return response.choices[0].message.content

            except Exception as e:
                print(f"Error: {e}")
            tries += 1

In [ ]:
WebAgent.openai_schema

### Agent with sequential tool calling

In [ ]:
class WebAgent(Agent):
    @function_schema
    def execute(self, query: Any) -> Any:
        """
        Ask the agent anything related to arithmetic or web search and it will try to answer it.
        """
        tries = 0
        system_prompt = self.system_prompt.prompt + f"""
        Here are the routing options:
        {"\n".join([f"{k}: {v}" for k, v in self.routing_options.items()])}

        Your response should be in the format:
        {{ "routing": "respond", "content": "The answer to the query."}}

        """
        messages=[{"role": "system", "content": system_prompt},{"role": "user", "content": query}]

        while tries < self.max_retries:
            print(tries)
            print(messages)
            try:
                response = client.chat.completions.create(
                    model="gpt-4o-mini",
                    messages=messages,
                    functions=self.functions,
                    # tools=self.functions,
                    # parallel_tool_calls=True,
                )
                print(response)
                if response.choices[0].message.function_call:
                    print(response.choices[0].message.function_call)
                    # print(**response.choices[0].message.function_call.arguments)
                    arguments = json.loads(response.choices[0].message.function_call.arguments)
                    function_name = response.choices[0].message.function_call.name
                    output = tool_store[function_name](**arguments)
                    # output = "Toshiba is a great company"
                    # print(output)
                    messages+=[{"role":"system", "content": f"I called the function {function_name} with the arguments {arguments} and got the output {output}."},
                        {"role": "user", "content": """Now, try to answer the original query given the output. If you can't, try using another tool. If it does answer the query, you can stop here and just answer the query. If you can't then you can give up.
                    """}]
                else:
                    return response.choices[0].message.content

            except Exception as e:
                print(f"Error: {e}")
            tries += 1

### Agent with parallel tool calling

In [ ]:
@agent_schema
class WebAgent(Agent):
    def execute(self, query: Any) -> Any:
        """
        Ask the agent anything related to arithmetic, customer orders numbers or web search and it will try to answer it.
        You can ask it multiple questions at once. No need to ask one question at a time. You can ask it for multiple customer ids, multiple arithmetic questions, or multiple web search queries.

        You can ask:
        query : what are the customer order numbers for customer id 1111 and 2222.
        query : what is the sum of 2 and 3, and sum of 12 and 13.
        query : what is the latest news on Toshiba and Apple.
        query : what is the sum of 12 and 13, and web results for "latest news on Toshiba.
        """
        tries = 0
        system_prompt = self.system_prompt.prompt + f"""
        Here are the routing options:
        {"\n".join([f"{k}: {v}" for k, v in self.routing_options.items()])}

        Your response should be in the format:
        {{ "routing": "respond", "content": "The answer to the query."}}

        """
        messages=[{"role": "system", "content": system_prompt},{"role": "user", "content": query}]

        while tries < self.max_retries:
            print(tries)
            # print("\n\nMessage: ",messages)
            try:
                response = client.chat.completions.create(
                    model="gpt-4o-mini",
                    messages=messages,
                    # functions=self.functions,
                    tools=self.functions,
                    # parallel_tool_calls=True,
                    tool_choice="auto",
                )
                print("\n\nResponse: ",response)
                if response.choices[0].finish_reason=="tool_calls":
                    tool_calls = response.choices[0].message.tool_calls
                    messages+=[{"role": "assistant", "tool_calls": tool_calls},]
                    for tool in tool_calls:
                        print(tool.function.name)
                        tool_id = tool.id
                        arguments = json.loads(tool.function.arguments)
                        function_name = tool.function.name
                        result = tool_store[function_name](**arguments)
                        print(result)
                        messages+= [{"role": "tool", "tool_call_id": tool_id, "content": str(result)}]

                else:
                    return response.choices[0].message.content

            except Exception as e:
                print(f"Error: {e}")
            tries += 1

In [ ]:
WebAgent.openai_schema

In [ ]:
import json

web_agent = WebAgent(name="WebSearchAgent",
                agent_id=uuid.uuid4(),
                system_prompt=agent_system_prompt,
                persona="Helper",
                functions=[web_search.openai_schema, add_numbers.openai_schema, get_customer_order.openai_schema],
                routing_options={"continue": "If you think you can't answer the query, you can continue to the next tool or do some reasoning.",
                                 "respond": "If you think you have the answer, you can stop here.",
                                 "give_up": "If you think you can't answer the query, you can give up and let the user know."
                                 },

                short_term_memory=True,
                long_term_memory=False,
                reasoning=False,
                input_type=["text", "voice"],
                output_type=["text", "voice"],
                response_type="json",
                max_retries=5,
                timeout=None,
                deployed=False,
                status="active",
                priority=None,
                failure_strategies=["retry", "escalate"],
                session_id=None,
                last_active=datetime.now(),
                collaboration_mode="single"
                )

In [ ]:
# web_agent.execute.openai_schema['function']['name'] = "web_agent"
# del web_agent.execute.openai_schema['function']['parameters']['properties']["self"]
# web_agent.execute.openai_schema['function']['parameters']['required'].pop(0)
# web_agent.execute.openai_schema

In [ ]:
agent_store = {
    "WebAgent": web_agent.execute
}

In [ ]:
# Add an execution method for the agent
res = web_agent.execute("Find the order number of customer ID 1111 and get the order number of customer of 2222. Find the sum of the order numbers, and search about the sum on the web. Tell me what you found on the web.")
# res = stock_agent.execute("What is the projected revenue for QBTS in 2025. Add its revenue with 195900?")
# res = stock_agent.execute("What is the latest news on Toshiba in March 2025?")
# res = stock_agent.execute("Prove Goldbach Conjecture")
print(res)


In [ ]:
res

In [ ]:
print(json.loads(res)["content"])

### Command Agent

In [ ]:
web_agent.openai_schema

In [ ]:
command_agent_system_prompt = PromptObject(pid=uuid.uuid4(),
                                                prompt_label="Command Agent",
                                                prompt="""You're a command agent. Assign proper tasks to the agents and look at their outputs to generate a final output. Try to ask multiple questions at once to the agent that is capable of doing so. For instance, if the agent is capable of doing three things, ask the agent to do the three things at once.

                                                Remeember, if an agent responds with the routing option "respond" then it means they are done with the task. If they respond with "continue" then they are not done with the task and you can ask them to do more tasks. If they respond with "give_up" then they are not able to do the task and you can ask another agent to do the task or tell the user you can't do it if the agents give up a lot of the times.
                                                """,
                                                sha_hash="000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8c",
                                                uniqueLabel="CommandAgent",
                                                appName="iopex",
                                                version="1.0",
                                                createdTime=datetime.now(),
                                                deployedTime=None,
                                                last_deployed=None,
                                                modelProvider="OpenAI",
                                                modelName="GPT-4o-mini",
                                                isDeployed=False,
                                                tags=["search", "web"],
                                                hyper_parameters={"temperature": "0.7"},
                                                variables={"search_engine": "google"})

In [ ]:
class CommandAgent(Agent):
    def execute(self, query: str) -> str:
        """
        This agent can call any agent, tool, or component. It can also call a router agent to call multiple agents.
        """
        tries = 0
        system_prompt = self.system_prompt.prompt

        messages=[{"role": "system", "content": system_prompt},{"role": "user", "content": query}]

        while tries < self.max_retries:
            print("\n\nCommand Agent Tries: ",tries)
            try:
                response = client.chat.completions.create(
                    model="o3-mini",
                    messages=messages,
                    # temperature=float(self.system_prompt.hyper_parameters["temperature"]),
                    # functions=self.functions,
                    tools=self.functions,
                    # parallel_tool_calls=True,
                    tool_choice="auto",
                )
                print("\n\nResponse: ",response)
                if response.choices[0].finish_reason=="tool_calls":
                    tool_calls = response.choices[0].message.tool_calls
                    messages+=[{"role": "assistant", "tool_calls": tool_calls},]
                    for tool in tool_calls:
                        print("\n\nAgent Called: ",tool)
                        tool_id = tool.id
                        arguments = json.loads(tool.function.arguments)
                        function_name = tool.function.name
                        result = agent_store[function_name](**arguments)
                        print(result)
                        messages+= [{"role": "tool", "tool_call_id": tool_id, "content": str(result)}]

                else:
                    return response.choices[0].message.content

            except Exception as e:
                print(f"Error: {e}")
            tries += 1


In [ ]:
web_command_agent = CommandAgent(name="WebCommandAgent",
                agent_id=uuid.uuid4(),
                system_prompt=command_agent_system_prompt,
                persona="Command Agent",
                functions=[web_agent.openai_schema],
                routing_options={"continue": "If you think you can't answer the query, you can continue to the next tool or do some reasoning.",
                                 "respond": "If you think you have the answer, you can stop here.",
                                 "give_up": "If you think you can't answer the query, you can give up and let the user know."
                                 },

                short_term_memory=True,
                long_term_memory=False,
                reasoning=False,
                input_type=["text", "voice"],
                output_type=["text", "voice"],
                response_type="json",
                max_retries=5,
                timeout=None,
                deployed=False,
                status="active",
                priority=None,
                failure_strategies=["retry", "escalate"],
                session_id=None,
                last_active=datetime.now(),
                collaboration_mode="single"
                )

In [ ]:
result_agent =web_command_agent.execute("Find the order number of customer ID 1111 and get the order number of customer of 2222. Find the sum of the order numbers, and search about the sum on the web. Tell me what you found on the web.")

In [ ]:
print(result_agent)

### APIs

In [6]:
from openai import OpenAI
import os
import dotenv
dotenv.load_dotenv(".env.local")
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

response = client.chat.completions.create(
    model="gpt-4o-mini",
    # response_format={"type": "json_schema", "json_schema": '{"animal_type": "animal name"}'},
    messages=[{
        "role": "user",
        "content": [
            {"type": "text", "text": "What's in this image?"},
            {
                "type": "image_url",
                "image_url": {
                    "url": "https://fastly.picsum.photos/id/237/200/300.jpg?hmac=TmmQSbShHz9CdQm0NkEjx1Dyh_Y984R9LpNrpvH2D_U",
                },
            },
        ],
    }],
)

print(response.choices[0].message.content)

The image shows a black puppy, likely a Labrador. It appears to be lying on a wooden surface, with a focused expression. The puppy has a shiny coat and big, expressive eyes.
